In [1]:
# Importar Bibliotecas
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon
import fiona
import rtree

# Município e Envelope

In [ ]:
# Importar e ler shapefile de município
shapefile= r'BR_Municipios_2021\BR_Municipios_2021.shp'

# Ler
mun_br = gpd.read_file(shapefile)

# Nome do município
nome = 'Piracicaba'

# Filtrar pelo nome do Município
mun = mun_br[mun_br.NM_MUN==f'{nome}']

In [ ]:
# Salvar shapefile do município na pasta selecionada
out=r'Shapefiles/municipio_sp.shp'
mun.to_file(out)


In [ ]:
# Visualizar shape
fig,eixo=plt.subplots(figsize=(10,10))
mun.plot(ax=eixo, color='red')

In [ ]:
# Extrair envelope
env = mun.envelope
bbox = mun.bounds
ly = float(bbox['maxy'] - bbox['miny'])
lx = float(bbox['maxx'] - bbox['minx'])

# Verificar se largura é maior que altura
    # Se sim, rotacionar 90° e escalar
if ly>lx:
    env90 = env.rotate(90)
    env = env90.scale(2.5,2)
else:
    env = env.scale(2.5,2)

    # Se não, aumentar escala em X e Y

In [ ]:
# Adicionar cabeçalho e transformar em df
envelope=gpd.GeoDataFrame()
envelope['geometry']=env
envelope.head()

# Processamento das Camadas

In [ ]:
# Importar shapefile das camadas
g=r'shapefiles/brasil_geologico_integrado.shp'
h=r'shapefiles/hidrogeologia_Brasil.shp'
p=r'shapefiles/Solos_5000.shp'
gm=r'shapefiles/geom_area.shp'

In [ ]:
# Ler camadas
geo=gpd.read_file(g)
hidro=gpd.read_file(h)
pedo=gpd.read_file(p)
geom=gpd.read_file(gm)

In [ ]:
# Verificar crs das camadas
geo.crs

In [ ]:
# Reprojetar camadas
geo=geo.to_crs(epsg=4674)
hidro=hidro.to_crs(epsg=4674)
geom=geom.to_crs(epsg=4674)
pedo=pedo.to_crs(epsg=4674)
env=env.to_crs(epsg=4674)

In [ ]:

# Verificar se os shapes estão no mapa
fig,eixo=plt.subplots(figsize=(10,10))
geom.plot(ax=eixo, color='green')
env.plot(ax=eixo)
mun.plot(ax=eixo, color='red')

In [ ]:
# Clipando as camadas
geo_clip = geo.clip(env)
hidro_clip = hidro.clip(env)
pedo_clip = pedo.clip(env)
geom_clip = geom.clip(env)

In [ ]:
# Clipar as camadas
geo_clip.to_crs(epsg=31983, inplace=True)
hidro_clip.to_crs(epsg=31983, inplace=True)
pedo_clip.to_crs(epsg=31983, inplace=True)
geom_clip.to_crs(epsg=31983, inplace=True)

In [ ]:
# Exportar as camadas na pasta do projeto
geo_clip.to_file(f'geologia_{nome}.shp')
hidro_clip.to_file(f'hidrogeo_{nome}.shp')
pedo_clip.to_file(f'pedologia_{nome}.shp')
geom_clip.to_file(f'geomorfo_{nome}.shp')